In [1]:
import pandas as pd
import re

# Load the original CSV
df = pd.read_csv("Multiple-choice Question with a Single Correct Answer.csv")

# Function to extract question and options A-E
def extract_mcq_components(row):
    # Combine question and prompt to catch all options (especially E)
    full_text = f"{row['question']}\n{row['question_prompt']}"
    
    # Split into question and options based on line breaks and A-E markers
    parts = re.split(r'\n(?=[A-E]\.)', full_text)
    question_main = parts[0].strip()
    options = {}

    # Extract each option (A to E)
    for part in parts[1:]:
        match = re.match(r'([A-E])\.\s*(.*)', part.strip())
        if match:
            options[match.group(1)] = match.group(2)

    # Return question + A-E options in order
    return pd.Series([question_main] + [options.get(opt, "") for opt in ['A', 'B', 'C', 'D', 'E']])

# Apply the function to the DataFrame
df[['question_cleaned', 'A', 'B', 'C', 'D', 'E']] = df.apply(extract_mcq_components, axis=1)

# Add a question prompt field (for prompting use)
df['question_prompt'] = "Select one best answer to the question below:"

# Keep only the relevant fields, and rename cleaned question column
final_df = df[['qid', 'youtube_url', 'question_cleaned', 'question_prompt', 'A', 'B', 'C', 'D', 'E']]
final_df.rename(columns={'question_cleaned': 'question'}, inplace=True)

# Save to a new CSV file
final_df.to_csv("mcq_cleaned.csv", index=False)

print("✅ Cleaning completed! Saved as 'mcq_cleaned.csv'")

✅ Cleaning completed! Saved as 'mcq_cleaned.csv'


C:\Users\reina\AppData\Local\Temp\ipykernel_9276\2648344996.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'question_cleaned': 'question'}, inplace=True)
